In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
#!pip install spotipy --upgrade

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')
song_attributes_df = pd.read_csv('../data/songAttributes_1999-2019.csv')

In [16]:
#spotify_qualities_df.info()
updated_song_attributes_df = song_attributes_df.drop(['Unnamed: 0', 'Popularity', 'Album'], axis=1)
updated_song_attributes_df = updated_song_attributes_df.rename(columns={'Name': 'song'})
updated_song_attributes_df.head(5)

,Acousticness,Artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,song,Speechiness,Tempo,TimeSignature,Valence
0,0.000728,Collective Soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,Welcome All Again,0.0309,106.022,4,0.365
1,0.018200,Collective Soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,Fuzzy,0.0282,120.027,4,0.408
2,0.000473,Collective Soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,Dig,0.0559,144.061,4,0.370
3,0.000970,Collective Soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,You,0.0254,111.975,4,0.183
4,0.000036,Collective Soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,My Days,0.0318,92.721,4,0.666


In [4]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['date'].dt.year
charts_df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021


In [5]:
lyrics_df.head()

,ranking,song,band_singer,songurl,titletext,url,year,lyrics,uri,danceability,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,1,Breathe,Faith Hill,/wiki/Breathe_(Faith_Hill_song),Breathe,/wiki/Faith_Hill,2000,I can feel the magic floating in the air\nBein...,spotify:track:3y4LxiYMgDl4RethdzpmNe,0.529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Smooth,Santana,/wiki/Smooth_(Santana_song),Smooth,/wiki/Santana_(band),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:0n2SEXB2qoRQg171q7XqeW,0.609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Smooth,Rob Thomas,/wiki/Smooth_(Santana_song),Smooth,/wiki/Rob_Thomas_(musician),2000,"Man, it's a hot one\nLike seven inches from th...",spotify:track:5IALWUYK0zDSEmZgb4ICvc,0.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Maria Maria,Santana,/wiki/Maria_Maria,Maria Maria,/wiki/Santana_(band),2000,"Ladies and gents, turn up your sound systems\n...",spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,Maria Maria,The Product G&B,/wiki/Maria_Maria,Maria Maria,/wiki/The_Product_G%26B,2000,Turn up this sound system\nTo the sound of Car...,spotify:track:3XKIUb7HzIF1Vu9usunMzc,0.777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  datetime64[ns]
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   year            330087 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(2)
memory usage: 18.9+ MB


In [7]:
charts_df['artist'].value_counts()

artist
Taylor Swift                                      1023
Elton John                                         889
Madonna                                            857
Drake                                              787
Kenny Chesney                                      769
                                                  ... 
RiceGum Featuring Alissa Violet                      1
XXXTENTACION Featuring PnB Rock & Trippie Redd       1
Justin Bieber Featuring Burna Boy                    1
Justin Bieber Featuring BEAM                         1
Frankie Vaughan                                      1
Name: count, Length: 10205, dtype: int64

In [8]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              620 non-null    object 
 1   Artist            620 non-null    object 
 2   Album             620 non-null    object 
 3   year              620 non-null    int64  
 4   Duration          620 non-null    int64  
 5   time_signature    620 non-null    int64  
 6   danceability      620 non-null    float64
 7   energy            620 non-null    float64
 8   key               620 non-null    int64  
 9   loudness          620 non-null    float64
 10  mode              620 non-null    int64  
 11  speechiness       620 non-null    float64
 12  Acousticness      620 non-null    float64
 13  Instrumentalness  620 non-null    float64
 14  liveness          620 non-null    float64
 15  valence           620 non-null    float64
 16  tempo             620 non-null    float64
 1

In [9]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [10]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            330087 non-null  datetime64[ns]
 1   rank            330087 non-null  int64         
 2   song            330087 non-null  object        
 3   artist          330087 non-null  object        
 4   last-week       297775 non-null  float64       
 5   peak-rank       330087 non-null  int64         
 6   weeks-on-board  330087 non-null  int64         
 7   year            330087 non-null  int32         
 8   datetime        330087 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int32(1), int64(3), object(2)
memory usage: 21.4+ MB


In [11]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100092 entries, 0 to 100091
Data columns (total 77 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   date              100092 non-null  datetime64[ns]
 1   rank_x            100092 non-null  int64         
 2   song              100092 non-null  object        
 3   artist            100092 non-null  object        
 4   last-week         94523 non-null   float64       
 5   peak-rank         100092 non-null  int64         
 6   weeks-on-board    100092 non-null  int64         
 7   year_x            100092 non-null  int32         
 8   datetime          100092 non-null  datetime64[ns]
 9   rank_y            100092 non-null  object        
 10  band_singer       100092 non-null  object        
 11  songurl           100092 non-null  object        
 12  titletext         100092 non-null  object        
 13  url               100092 non-null  object        
 14  year

In [12]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

KeyError: 'year'

In [ ]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [ ]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
#artists

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

In [ ]:
spotify_qualities_df.dtypes

In [ ]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

In [ ]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

In [ ]:
spotify_qualities_df['year'].max()
charts_and_lyrics_df['year'].max()

In [ ]:
charts_and_lyrics_df.info()

In [ ]:
updated_charts_and_lyrics_df = no_repeats_range_charts_and_lyrics_df.combine_first(spotify_qualities_df.set_index('song')).reset_index()
updated_charts_and_lyrics_df.head()